### Prediction of kcr sites in non-histone proteins. The threshold is 0.5.  

In [19]:
from Bio import SeqIO
import numpy as np
from keras import backend as K
import tensorflow as tf
from keras import initializers,layers,regularizers
from keras.layers import Dropout
from keras import callbacks
from keras.models import *
from keras.engine.topology import Layer
from keras.callbacks import EarlyStopping
from keras.layers import Dropout,Activation
from keras import optimizers
from sklearn.model_selection import train_test_split
import keras
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn import metrics
import matplotlib.pyplot as plt
from matplotlib import pyplot
from matplotlib import pyplot as plt
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import auc
from scipy import interp
from keras.regularizers import l2
from keras.layers import Dense,Input,Activation,Conv1D,BatchNormalization,Concatenate
from sklearn.manifold import TSNE
from sklearn.utils import class_weight
import sys

In [20]:
keras.__version__

'2.2.4'

In [21]:
sys.version

'3.7.4 (default, Aug 13 2019, 20:35:49) \n[GCC 7.3.0]'

### To read the fasta file

In [22]:
def read_fasta(file_path):
    one=list(SeqIO.parse(file_path,'fasta'))
    return one

### To encode the amino acid using popular one-hot encoding

In [23]:
def onehot(seq):
    bases = ['A','C','D','E','F','G','H','I','K','L','M','N','P','Q','R','S','T','V','W','Y']
    X = np.zeros((len(seq),len(seq[0]),len(bases)))
    for i,m in enumerate(seq):
        for l,s in enumerate(m):
    #         print(s)
            if s in bases:
                X[i,l,bases.index(s)] = 1
    return X

### Proposed capsnet model core code stares from here


### Squash Activation

####  
    The non-linear activation used in Capsule. It drives the length of a large vector to near 1 and small vector to 0
    :param vectors: some vectors to be squashed, N-dim tensor
    :param axis: the axis to squash
    :return: a Tensor with same shape as input vectors

In [24]:
def squash(vectors, axis=-1):
    s_squared_norm = K.sum(K.square(vectors), axis, keepdims=True)
    scale = s_squared_norm / (1 + s_squared_norm) / K.sqrt(s_squared_norm + K.epsilon())
    return scale * vectors

In [25]:
class Length(layers.Layer):
    def call(self, inputs, **kwargs):
        return K.sqrt(K.sum(K.square(inputs), -1))
    def compute_output_shape(self, input_shape):
        return input_shape[:-1]

### Capsule Layer
The capsule layer. It is similar to Dense layer. Dense layer has `in_num` inputs, each is a scalar, the output of the
    neuron from the former layer, and it has `out_num` output neurons. CapsuleLayer just expand the output of the neuron
    from scalar to vector. So its input shape = [None, input_num_capsule, input_dim_vector] and output shape = \
    [None, num_capsule, dim_vector]. For Dense Layer, input_dim_vector = dim_vector = 1.
    :param num_capsule: number of capsules in this layer
    :param dim_vector: dimension of the output vectors of the capsules in this layer
    :param num_routings: number of iterations for the routing algorithm

In [26]:
class CapsuleLayer(layers.Layer):
    def __init__(self, num_capsule, dim_vector, num_routing=3,
                 kernel_initializer='glorot_uniform',
                 bias_initializer='zeros',
                 **kwargs):
        super(CapsuleLayer, self).__init__(**kwargs)
        self.num_capsule = num_capsule
        self.dim_vector = dim_vector
        self.num_routing = num_routing
        self.kernel_initializer = initializers.get(kernel_initializer)
        self.bias_initializer = initializers.get(bias_initializer)

    def build(self, input_shape):
        assert len(input_shape) >= 3, "The input Tensor should have shape=[None, input_num_capsule, input_dim_vector]"
        self.input_num_capsule = input_shape[1]
        self.input_dim_vector = input_shape[2]

        # Transform matrix
        self.W = self.add_weight(
            shape=[self.input_num_capsule, self.num_capsule, self.input_dim_vector, self.dim_vector],
            initializer=self.kernel_initializer,
            name='W')

        # Coupling coefficient. The redundant dimensions are just to facilitate subsequent matrix calculation.
        self.bias = self.add_weight(shape=[1, self.input_num_capsule, self.num_capsule, 1, 1],
                                    initializer=self.bias_initializer,
                                    name='bias',
                                    trainable=False)
        self.built = True
    def call(self, inputs, training=None):
        # inputs.shape=[None, input_num_capsule, input_dim_vector]
        # Expand dims to [None, input_num_capsule, 1, 1, input_dim_vector]
        inputs_expand = K.expand_dims(K.expand_dims(inputs, 2), 2)

        # Replicate num_capsule dimension to prepare being multiplied by W
        # Now it has shape = [None, input_num_capsule, num_capsule, 1, input_dim_vector]
        inputs_tiled = K.tile(inputs_expand, [1, 1, self.num_capsule, 1, 1])

        """
        # Begin: inputs_hat computation V1 ---------------------------------------------------------------------#
        # Compute `inputs * W` by expanding the first dim of W. More time-consuming and need batch_size.
        # w_tiled.shape = [batch_size, input_num_capsule, num_capsule, input_dim_vector, dim_vector]
        w_tiled = K.tile(K.expand_dims(self.W, 0), [self.batch_size, 1, 1, 1, 1])
        # Transformed vectors, inputs_hat.shape = [None, input_num_capsule, num_capsule, 1, dim_vector]
        inputs_hat = K.batch_dot(inputs_tiled, w_tiled, [4, 3])
        # End: inputs_hat computation V1 ---------------------------------------------------------------------#
        """

        # Begin: inputs_hat computation V2 ---------------------------------------------------------------------#
        # Compute `inputs * W` by scanning inputs_tiled on dimension 0. This is faster but requires Tensorflow.
        # inputs_hat.shape = [None, input_num_capsule, num_capsule, 1, dim_vector]
        inputs_hat = tf.scan(lambda ac, x: K.batch_dot(x, self.W, [3, 2]),
                             elems=inputs_tiled,
                             initializer=K.zeros([self.input_num_capsule, self.num_capsule, 1, self.dim_vector]))
        # End: inputs_hat computation V2 ---------------------------------------------------------------------#
        """
        # Begin: routing algorithm V1, dynamic ------------------------------------------------------------#
        def body(i, b, outputs):
            c = tf.nn.softmax(b, dim=2)  # dim=2 is the num_capsule dimension
            outputs = squash(K.sum(c * inputs_hat, 1, keepdims=True))
            if i != 1:
                b = b + K.sum(inputs_hat * outputs, -1, keepdims=True)
            return [i-1, b, outputs]
        cond = lambda i, b, inputs_hat: i > 0
        loop_vars = [K.constant(self.num_routing), self.bias, K.sum(inputs_hat, 1, keepdims=True)]
        shape_invariants = [tf.TensorShape([]),
                            tf.TensorShape([None, self.input_num_capsule, self.num_capsule, 1, 1]),
                            tf.TensorShape([None, 1, self.num_capsule, 1, self.dim_vector])]
        _, _, outputs = tf.while_loop(cond, body, loop_vars, shape_invariants)
        # End: routing algorithm V1, dynamic ------------------------------------------------------------#
        """

        # Begin: routing algorithm V2, static -----------------------------------------------------------#
        # Routing algorithm V2. Use iteration. V2 and V1 both work without much difference on performance
        assert self.num_routing > 0, 'The num_routing should be > 0.'
        for i in range(self.num_routing):
            c = tf.nn.softmax(self.bias, dim=2)
            
            # dim=2 is the num_capsule dimension
            # outputs.shape=[None, 1, num_capsule, 1, dim_vector]
            outputs = squash(K.sum(c * inputs_hat, 1, keepdims=True))

            # last iteration needs not compute bias which will not be passed to the graph any more anyway.
            if i != self.num_routing - 1:
                # self.bias = K.update_add(self.bias, K.sum(inputs_hat * outputs, [0, -1], keepdims=True))
                self.bias += K.sum(inputs_hat * outputs, -1, keepdims=True)
                # tf.summary.histogram('BigBee', self.bias)  # for debugging
        # End: routing algorithm V2, static ------------------------------------------------------------#

        return K.reshape(outputs, [-1, self.num_capsule, self.dim_vector])

    def compute_output_shape(self, input_shape):
        return tuple([None, self.num_capsule, self.dim_vector])
    

### PrimeryCap

In [27]:
def PrimaryCap(inputs, dim_vector, n_channels, kernel_size, strides, padding,dropout):
    output = layers.Conv1D(filters=dim_vector * n_channels, kernel_size=kernel_size, strides=strides, padding=padding, 
                           name='primarycap_conv1d')(inputs)
    outputs = layers.Reshape(target_shape=[-1, dim_vector], name='primarycap_reshape')(output)
    return layers.Lambda(squash, name='primarycap_squash')(outputs)

In [28]:
def CapsNet(input_shape, n_class, num_routing):
    x = layers.Input(shape=input_shape)
    conv1 = layers.Conv1D(filters=32, kernel_size=7, strides=1, padding='valid', activation='relu', name='conv1')(x)
    conv1=Dropout(0.7)(conv1)
    primarycaps = PrimaryCap(conv1, dim_vector=8, n_channels=16, kernel_size=7, strides=1, padding='valid',dropout=0.2)
    KcrCaps = CapsuleLayer(num_capsule=n_class, dim_vector=8, num_routing=num_routing,kernel_initializer='he_normal', name='KcrCaps')(primarycaps)
    out = Length(name='capsnet')(KcrCaps)
    train_model = Model(x, out)
    return train_model

### CapsNh-Kcr 

In [29]:
mode1= CapsNet(input_shape=(29,20),n_class=2,num_routing=3)
mode1.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 29, 20)            0         
_________________________________________________________________
conv1 (Conv1D)               (None, 23, 32)            4512      
_________________________________________________________________
dropout_2 (Dropout)          (None, 23, 32)            0         
_________________________________________________________________
primarycap_conv1d (Conv1D)   (None, 17, 128)           28800     
_________________________________________________________________
primarycap_reshape (Reshape) (None, 272, 8)            0         
_________________________________________________________________
primarycap_squash (Lambda)   (None, 272, 8)            0         
_________________________________________________________________
KcrCaps (CapsuleLayer)       (None, 2, 8)              35360     
__________

### Upload a data for prediction without labeling the data.  In our case,  here, we used the independent data (as described in the manuscript)  for prediction.  

In [36]:
all_data= read_fasta('Kcr_IND.txt')

In [31]:
def get_pred():
    positive_ind_data=positive_ind_data=all_data[0:3343] # positive data # use read_rasta function written in the beginning
    print("The total length of positive sequences to be predicted is:", len(positive_ind_data))
    negative_ind_data=all_data[3344:6687] #negative data
    print("The total length of negative sequences to be predicted is:", len(negative_ind_data))
    all_ind_data=positive_ind_data+negative_ind_data
    all_ind_onehot=onehot(all_ind_data) # use onehot function  written in the beginning
    all_pred_y=[]
    for i in range(5):
        mode1.load_weights(str(i+0)+'nonhistoneLastepoch_new'+'.h5')
        pred_y=mode1.predict(all_ind_onehot)
        #all_pred_y.append(pred_y)
        pred_y=np.argmax(pred_y, axis=1)
        all_pred_y.append(pred_y)
        print(pred_y)
    all_pred_y=np.average(all_pred_y,axis=0)
    print(all_pred_y)
    return all_pred_y

In [32]:
result=get_pred()

The total length of positive sequences to be predicted is: 3343
The total length of negative sequences to be predicted is: 3343
[1 1 1 ... 0 0 0]
[1 1 1 ... 0 0 0]
[1 1 1 ... 0 0 0]
[1 1 1 ... 0 1 0]
[1 1 1 ... 0 0 0]
[1.  1.  1.  ... 0.  0.2 0. ]


### Prediction of first 500 sequence (positive sequence or kcr site containing sequences)

In [33]:
print("The result shows prediction for first 500 positive sequence :",' \n', result[:500])

The result shows prediction for first 1000 positive sequence :  
 [1.  1.  1.  1.  1.  1.  1.  1.  1.  0.4 0.  1.  1.  0.  1.  1.  0.  1.
 0.4 0.2 1.  1.  1.  0.6 1.  1.  0.6 0.  1.  0.8 1.  1.  0.  1.  1.  1.
 1.  1.  1.  1.  0.  1.  1.  1.  1.  1.  0.6 1.  1.  1.  1.  1.  1.  0.2
 1.  0.6 1.  1.  1.  0.  0.  0.  0.6 0.  0.2 1.  1.  1.  1.  0.4 1.  1.
 1.  1.  1.  1.  1.  1.  1.  0.2 0.  1.  1.  1.  1.  1.  0.  1.  1.  1.
 1.  1.  0.  1.  1.  1.  1.  1.  1.  1.  0.8 1.  1.  1.  1.  0.6 1.  1.
 1.  1.  1.  1.  1.  1.  1.  0.2 1.  0.  0.4 1.  1.  1.  1.  1.  1.  1.
 1.  1.  1.  1.  1.  0.  1.  1.  1.  0.  1.  0.8 0.4 1.  1.  0.2 0.8 1.
 1.  1.  1.  1.  0.8 1.  1.  1.  1.  1.  0.2 0.2 1.  1.  1.  1.  1.  1.
 1.  1.  0.  1.  1.  1.  1.  0.  1.  1.  1.  1.  0.8 1.  1.  0.6 1.  0.
 1.  1.  1.  1.  1.  1.  0.  1.  1.  1.  1.  1.  0.2 0.  1.  1.  1.  1.
 1.  0.8 1.  1.  1.  0.6 1.  0.  1.  1.  1.  1.  1.  1.  0.6 1.  1.  1.
 0.  1.  1.  1.  1.  0.6 0.  1.  1.  1.  0.6 0.  1.  1.  1.  0.  0.  

### Prediction of last 500 negative samples

In [35]:
print("The result shows prediction for last 500 negative sequence :",' \n', result[-500:])

The result shows prediction for last 500 negative sequence :  
 [0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.4 0.4 0.  0.  0.  0.  0.
 0.  0.  1.  0.  0.  0.  1.  0.  0.  0.4 0.  0.  0.  0.  0.  0.  1.  1.
 0.  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.2 1.  0.  1.  0.  0.  0.2 0.
 0.  0.  0.2 0.2 0.  1.  0.  1.  1.  0.  1.  0.  0.  0.  1.  0.  0.  0.
 0.2 0.  0.  0.  0.  0.  1.  0.  1.  0.  0.  0.  0.  0.  0.  1.  0.  1.
 0.  1.  0.  1.  0.  1.  1.  0.6 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.2
 0.8 0.  0.  0.6 0.  1.  1.  0.  0.8 0.  1.  0.  1.  0.  1.  0.4 1.  0.
 0.  0.  1.  0.  0.  1.  0.  0.  1.  0.  0.8 0.  1.  0.  1.  0.  0.4 0.
 0.  0.  0.  1.  0.  0.  0.  1.  0.  1.  0.  1.  0.  0.  0.2 0.  0.  0.
 0.  0.  0.  0.  1.  1.  1.  0.8 0.  0.  0.  0.  0.  0.  1.  1.  1.  0.
 1.  0.8 0.6 0.  0.  0.2 0.4 0.2 1.  0.  1.  0.  0.  0.  0.  0.8 1.  0.
 0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  1.  0.  0.  1.  0.  1.
 1.  0.  1.  0.  0.  1.  0.  0.  0.  0.  1.  0.  0.  0.  0.  0.6 0.  0.